## Forward Propagation in simple steps

- Code for loading libraries has been skipped as of now. 
- I'm only focussing on how layers are created and activations calculated for subsequent layers.

In [ ]:
#Define data

#temperature and duration for roasting coffee beans
xtrain = np.array([[200.0, 17]])

### Implementing forward prop one layer at a time

The network defined here has only 2 layers- one hidden layer and one output layer.  
Layer 1 has 3 neurons with sigmoid as activation function.

In [ ]:
#Layer 1
layer_1 = Dense(units = 3, activation = 'sigmoid')
#calculate output of layer 1
a1 = layer_1(xtrain)

Layer 2 has 1 neuron with sigmoid as activation function.

In [ ]:
#Layer 2
layer_2 = Dense(units = 1, acitivation = 'sigmoid')
a2 = layer_2(a1)

In [ ]:
#Making binary inference from output
if a2 >= 0.5:
    yhat = 1
else:
    yhat = 0

### Implementing forward propagation- short version

In [ ]:
#Define data
#temperature and duration for coffee beans roasting
x = np.array([[200.0, 17.0],
              [120.0, 5.0],
              [425.0, 20.0],
              [212.0, 18.0]])
#good coffee-bad coffee labels
y = np.array([1,0,0,1])

`Sequential()` strings all layers together in the given order.  
In this way, we do not need to explicitly pass output of one layer to the next. This gets handled by Tensorflow itself.

In [ ]:
#Define layers
layer_1 = Dense(units = 3, activation = 'sigmoid')
layer_2 = Dense(units = 1, activation = 'sigmoid')
#create the mode
model = Sequential([layer_1, layer_2])

In [ ]:
#A more compact version
model = Sequential([
    Dense(units = 3, activation = 'sigmoid', name = 'layer_1'),
    Dense(units = 1, activation = 'sigmoid', name = 'layer_2')
])

In [ ]:
#model.compile() defines a loss function and compile optimization
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    x,y,            
    epochs=10,
)

#make inference on new data
model.predict(x_test)

In [ ]:
#View the updated weights
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

### Steps in data preprocessing

#### Normalization layer

The procedure below uses a Keras [normalization layer](https://keras.io/api/layers/preprocessing_layers/numerical/normalization/). It has the following steps:
- create a "Normalization Layer". Note, this is not a layer in your model.
- 'adapt' the data. This learns the mean and variance of the data set and saves the values internally.
- normalize the data.  
It is important to apply normalization to any future data that utilizes the learned model.

In [ ]:
#create normalization layer
norm_l = tf.keras.layers.Normalization(axis=-1)
#learn the mean and variance of each column feature
norm_l.adapt(X)  # learns mean, variance
#normalize all column of the data
Xn = norm_l(X)

In [ ]:
#getting weights and biases
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

In [ ]:
### Normalizing test data and then making predictions
X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

To convert the probabilities to a decision, we apply a threshold:

In [ ]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

In [ ]:
#compact version of above code chunk
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")